In [ ]:
#Import tensorflow and checking the versions
import tensorflow as tf
print(tf.__version__)

2.8.2


In [ ]:
!nvidia-smi

Sun Jul 10 05:21:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    32W /  70W |  11548MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

##Extracting the Dataset

In [ ]:
!unzip 'train.csv.zip'
!unzip 'test.csv.zip'

In [ ]:
!rm -rf test.csv.zip
!rm -rf train.csv.zip

##Importing important packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import tensorflow as tf
# Input data files are available in the "../input/" directory.
from subprocess import check_output
print(check_output(["ls", "../content"]).decode("utf8"))

sample_data
test.csv
train.csv



#Read and Reshape the dataset

In [ ]:
# Read training and test data files
train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

In [ ]:
print(train.shape)
print(test.shape)

(42000, 785)
(28000, 784)


In [ ]:
train_y = train['label'].astype('float32')
train_x = train.drop(['label'],axis = 1).astype('int32')
test_x = test.astype('float32')
train_x.shape, train_y.shape, test_x.shape

((42000, 784), (42000,), (28000, 784))

In [ ]:
train_x = train_x.values.reshape(-1,28,28,1)
train_x = train_x / 255.0
test_x = test_x.values.reshape(-1,28,28,1)
test_x = test_x / 255.0
train_x.shape, test_x.shape

((42000, 28, 28, 1), (28000, 28, 28, 1))

In [ ]:
train_x = np.concatenate((train_x, train_x.reshape(-1, 28,28, 1)))
train_y = np.concatenate((train_y, train_y))

In [ ]:
#One Hot Encoding
train_y = tf.keras.utils.to_categorical(train_y,10)
train_y.shape

(84000, 10)

In [ ]:
#Printing original labels of top 5 rows
print(train['label'])

In [ ]:
#One hot encoding of the same labels
print(train_y[,:])

In [ ]:
#Defining the model
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(32,(3,3),activation = 'relu', input_shape=(28,28,1)),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(32,(3,3),activation = 'relu'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'Same'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'Same'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'Same'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.Conv2D(64,(3,3),activation = 'relu',padding = 'Same'),
  tf.keras.layers.BatchNormalization(),
  tf.keras.layers.MaxPooling2D(pool_size = (2,2), strides = (2,2)),
  tf.keras.layers.Dropout(0.25),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.50),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 24, 24, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 32)       0         
 )                                                               
                                                        

In [ ]:
#Defining the callback function to stop our training once the acceptable accuracy is reached
class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if(logs.get('accuracy') > 0.9999):
                print("\nReached 99.9% accuracy so cancelling training!")
                self.model.stop_training = True
    
callbacks = myCallback()

In [ ]:
train_x.shape

(84000, 28, 28, 1)

In [ ]:
#Compiling and model training with batch size = 50, epochs = 20, and optimizer = adam
Optimizer = tf.keras.optimizers.Adam(
            learning_rate=0.001, 
            beta_1=0.9, 
            beta_2=0.999, 
            epsilon=1.0,
            name='Adam'
)
model.compile(optimizer=Optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_x, train_y, batch_size = 48, epochs = 20, callbacks=[callbacks])

Epoch 1/20
1750/1750 [==============================] - 12s 6ms/step - loss: 0.0082 - accuracy: 0.9980
Epoch 2/20
1750/1750 [==============================] - 11s 6ms/step - loss: 0.0078 - accuracy: 0.9982
Epoch 3/20
1750/1750 [==============================] - 11s 6ms/step - loss: 0.0066 - accuracy: 0.9982
Epoch 4/20
1750/1750 [==============================] - 11s 6ms/step - loss: 0.0070 - accuracy: 0.9983
Epoch 5/20
1750/1750 [==============================] - 11s 6ms/step - loss: 0.0053 - accuracy: 0.9985
Epoch 6/20
1750/1750 [==============================] - 11s 6ms/step - loss: 0.0061 - accuracy: 0.9984
Epoch 7/20
1750/1750 [==============================] - 11s 6ms/step - loss: 0.0054 - accuracy: 0.9985
Epoch 8/20
1750/1750 [==============================] - 11s 6ms/step - loss: 0.0056 - accuracy: 0.9985
Epoch 9/20
1750/1750 [==============================] - 11s 6ms/step - loss: 0.0049 - accuracy: 0.9986
Epoch 10/20
1750/1750 [==============================] - 11s 6ms/step - l

In [ ]:
results = model.predict(test_x)

# select the index with the maximum probability
results = np.argmax(results,axis = 1)

results = pd.Series(results,name="Label")

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("submission1.csv",index=False)